In [1]:
from tqdm.auto import tqdm

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from quadprog import solve_qp #https://pypi.org/project/quadprog/ || installed with conda -- look into how/what || #dual method of Goldfarb and Idnani (1982, 1983) for solving quadratic programming problems of the form 
from joblib import Parallel, delayed

def perform_ols(sig_mat, bulk_sample):
    # NOTES:
    # S = signature matrix ==> sig_mat
    # B = Bulk data individual sample ==> bulk_sample
    # sol = OLS initial weights ==> ols_weights
    sigsig_dot = sig_mat.T.dot(sig_mat).values #D=G
    sigbulk_dot = sig_mat.T.dot(bulk_sample).values #d=a
    sig_eye_mat = np.identity(sig_mat.shape[1]) #A=C
    sig_zeros_mat = np.array([0.0] * sig_mat.shape[1]) #b=x ; TODO: lookup the purpose? ; also must be float: https://stackoverflow.com/questions/36510859/cvxopt-qp-solver-typeerror-a-must-be-a-d-matrix-with-1000-columns
    qprog_sol = solve_qp(G=sigsig_dot, a=sigbulk_dot, C=sig_eye_mat, b=sig_zeros_mat, meq=0, factorized=False)
    qprog_sol = qprog_sol[0] #gets the solution
    qprog_sol = pd.DataFrame(qprog_sol, index=sig_mat.columns) #convert to dataframe, with cell type as index
    
    return qprog_sol

def find_dampening_constant(sig_mat, bulk_sample, ols_weights, epochs=100):
    ws = (1/sig_mat.dot(ols_weights)).pow(2)
    ws_scaled = ws/ws.min()
    ws_val = int(np.ceil(np.log2(ws_scaled.loc[ws_scaled[0] != np.inf, 0].max())))

    solutions_std_list = []
    for j in range(ws_val):
        multiplier = np.power(2, j)
        ws_dampened = ws_scaled.copy()
        ws_dampened.loc[ws_dampened[0] > multiplier, 0] = multiplier
        
        solutions = []
        for i in range(epochs):
            np.random.seed(i)
            n=int(ws_dampened.shape[0]/2)
            subset_indices = np.random.choice(a=ws_dampened.shape[0], size=n, replace=False)
            y = bulk_sample.iloc[subset_indices]
            X = sig_mat.iloc[subset_indices] #Having -1 adds a constant, purpose?IDK: https://stackoverflow.com/questions/52596724/linear-regression-in-r-and-python-different-results-at-same-problem
            w = ws_dampened.iloc[subset_indices]
            wls_model = sm.WLS(y, X, weights=w, missing="raise")
            wls_result = wls_model.fit(method="qr").params
            wls_result = wls_result * (ols_weights.sum() / wls_result.sum()).values[0]
            solutions.append(wls_result)


        solutions_std = pd.concat(solutions, axis=1).std(axis=1)
        solutions_std = solutions_std.rename(j+1) #rename to dampening constant (multiplier)+1
        solutions_std_list.append(solutions_std)

    solutions_std_df = pd.concat(solutions_std_list, axis=1)
    dampening_constant = solutions_std_df.pow(2).mean(axis=0).idxmin() #index=dampening constant (multiplier)
    return dampening_constant

def solve_dampenedWLS_with_constant(sig_mat, bulk_sample, ols_weights, dampening_constant):
    # NOTES:
    # S = signature matrix ==> sig_mat
    # B = Bulk data individual sample ==> bulk_sample
    # sol = goldStandard = OLS initial weights ==> ols_weights
    multiplier = 1 * np.power(2, (dampening_constant-1))
    sol = ols_weights.copy()
    ws = (1 / sig_mat.dot(sol)).pow(2)
    ws_scaled = ws / ws.min()
    ws_dampened = ws_scaled.copy()
    ws_dampened.loc[ws_dampened[0] > multiplier, 0] = multiplier
    W_diag = pd.DataFrame(np.diag(ws_dampened[0]),index=ws_dampened.index,columns=ws_dampened.index)
    D = sig_mat.T.dot(W_diag).dot(sig_mat)
    d = sig_mat.T.dot(W_diag).dot(bulk_sample)
    sig_eye_mat = np.identity(sig_mat.shape[1]) #A
    sig_zeros_mat = np.array([0.0] * sig_mat.shape[1]) #b=x ; TODO: lookup the purpose? ; also must be float: https://stackoverflow.com/questions/36510859/cvxopt-qp-solver-typeerror-a-must-be-a-d-matrix-with-1000-columns
    sc = np.linalg.norm(D, ord=2) #2-norm (largest sing. value) || specifies the “spectral” or 2-norm, which is the largest singular value (svd) of x.
    qprog_sol = solve_qp(G=(D/sc).values, a=(d/sc).values, C=sig_eye_mat, b=sig_zeros_mat, meq=0, factorized=False)
    qprog_sol = qprog_sol[0] #gets the solution
    qprog_sol = pd.DataFrame(qprog_sol, index=sig_mat.columns) #convert to dataframe, with cell type as index
    return qprog_sol

def solve_dampenedWLS(sig_mat, bulk_sample):
    ols_weights = perform_ols(sig_mat, bulk_sample)
    dampening_constant = find_dampening_constant(sig_mat, bulk_sample, ols_weights, epochs=100)
    iterations =0
    changes = []
    change = 1
    solution = solve_dampenedWLS_with_constant(sig_mat, bulk_sample, ols_weights, dampening_constant)
    while change>.01 and iterations<1000 : 
        new_solution = solve_dampenedWLS_with_constant(sig_mat, bulk_sample, ols_weights, dampening_constant)
        solution_average = pd.concat([solution] + [new_solution]*4 , axis=1).mean(1)
        change = np.linalg.norm((solution_average-solution[0]).to_frame(), ord=1)
        solution = solution_average
        iterations += 1
        changes.append(change)
        
    return solution/solution.sum()

In [3]:
#USER INPUT TO DO DWLS
signature_matrix_file_location = "SIGNATURE MATRIX FILE LOCATION"
signature_matrix_delimiter ='DELIMITER'

bulk_data_file_location = "BULK DATA FILE LOCATION"
bulk_data_delimiter ='DELIMITER'

In [5]:
#Assume first column is gene markers || have a condition check in #intersection of sig and bulk data
signature_matrix = pd.read_csv(signature_matrix_file_location, sep=signature_matrix_delimiter, index_col=0)
signature_matrix.index = signature_matrix.index.str.upper()

bulk_data = pd.read_csv(bulk_data_file_location, sep=bulk_data_delimiter, index_col=0)
bulk_data.index = bulk_data.index.str.upper()

#Filter matrices by similar genes
sig_mat_genes = set(signature_matrix.index.str.upper())
bulk_genes = set(bulk_data.index.str.upper())
genes_intersecting = sig_mat_genes.intersection(bulk_genes)

#Get signature matrix and get intersecting genes
sig_mat = signature_matrix #get fresh matrix with each loop #Is this really needed, as im not changing the sig mat?
sig_mat = sig_mat.loc[genes_intersecting]

#Get bulk data with intersecting genes
bulk_data = bulk_data.loc[genes_intersecting]

dwls_solutions = Parallel(n_jobs=-1)(delayed(solve_dampenedWLS)(sig_mat, bulk_data[col]) for col in tqdm(bulk_data.columns))
dwls_solutions = pd.concat(dwls_solutions, axis=1)
dwls_solutions

,0,1,2,3,4,5,6,7,8,9,...,936,937,938,939,940,941,942,943,944,945
gabaergic|7,8.487162e-02,8.449961e-02,9.760995e-02,7.972375e-02,1.105385e-01,7.328533e-02,1.066898e-01,8.127332e-02,1.057648e-01,7.794498e-02,...,6.440964e-02,8.905118e-02,8.128565e-02,6.662295e-02,7.551103e-02,1.335238e-01,1.265518e-01,1.206782e-01,1.224731e-01,1.376228e-01
gabaergic|5,1.354561e-02,9.868649e-19,4.908578e-02,6.928321e-02,0.000000e+00,5.374825e-02,4.632939e-02,2.657198e-02,3.765776e-02,-7.039280e-18,...,0.000000e+00,2.693894e-02,3.674158e-02,0.000000e+00,1.382410e-02,9.656837e-02,1.052536e-01,9.510026e-02,1.126899e-01,9.511235e-02
gabaergic|4,7.773870e-18,5.707128e-02,-1.887173e-18,1.481348e-17,2.765913e-17,3.284921e-17,7.235077e-20,-6.530802e-18,-1.658618e-17,1.120234e-18,...,-9.254687e-18,-6.308358e-17,2.355207e-17,1.006077e-17,-1.620262e-17,9.366170e-18,4.055404e-17,3.107011e-18,1.142616e-17,4.197853e-18
gabaergic|3,0.000000e+00,1.778450e-18,3.292900e-18,-3.238135e-19,-2.747794e-18,4.101186e-20,5.825498e-20,0.000000e+00,8.050719e-18,7.704123e-18,...,1.646987e-17,0.000000e+00,0.000000e+00,-5.109094e-18,1.950174e-20,4.168409e-19,-5.272104e-19,2.898161e-18,2.926857e-19,-1.355206e-18
gabaergic|10,5.456262e-17,-4.518626e-17,-2.700023e-17,-1.558524e-16,-1.262005e-17,1.460083e-16,6.787634e-17,-1.332731e-18,-3.305430e-17,7.298200e-17,...,6.387549e-17,7.366049e-18,-1.039297e-17,9.107598e-18,-2.121761e-17,-1.522127e-18,6.896427e-17,1.194321e-16,1.119016e-16,-1.057895e-16
gabaergic|12,6.017078e-02,4.068632e-02,4.414689e-02,4.132331e-02,8.441319e-02,3.124965e-02,5.870113e-02,6.160352e-02,5.576158e-02,4.825405e-02,...,1.006803e-01,5.741490e-02,6.538154e-02,8.910087e-02,7.855152e-02,8.498913e-03,7.473628e-03,1.390357e-02,1.107795e-02,0.000000e+00
glutamatergic|9,-4.851228e-18,-2.627642e-17,6.394773e-17,2.734932e-19,1.470561e-17,-4.218733e-17,-5.974039e-18,2.027510e-17,-3.738582e-17,-4.339198e-18,...,-2.566455e-18,-1.045256e-17,-7.029030e-18,-1.777479e-18,-3.612337e-18,2.340909e-17,3.239094e-17,-1.861084e-17,-2.980763e-18,3.244665e-18
glutamatergic|13,8.123843e-02,1.038850e-01,5.860221e-02,2.470999e-02,9.947907e-02,2.487539e-01,4.627044e-03,2.602547e-02,9.859348e-03,1.078010e-01,...,6.211653e-02,1.147536e-01,1.112192e-01,6.190738e-02,1.055497e-01,-1.238230e-18,5.512165e-18,0.000000e+00,0.000000e+00,-6.918963e-19
glutamatergic|11,2.961255e-02,1.683149e-02,2.992529e-02,5.143219e-02,5.946621e-02,2.389250e-02,5.243711e-02,7.266538e-02,5.492075e-02,5.339041e-02,...,9.297965e-02,2.088500e-02,3.012065e-02,8.127056e-02,2.283876e-02,1.057521e-01,1.088010e-01,1.184870e-01,1.101175e-01,1.322493e-01
glutamatergic|21,9.812425e-17,2.491556e-16,-9.357292e-17,1.412037e-16,1.998198e-16,9.149809e-17,7.116463e-17,1.506739e-16,-1.454330e-16,-4.119513e-16,...,-7.353427e-17,2.026351e-16,2.159759e-16,-1.838382e-16,-4.230077e-17,-7.203522e-18,9.223841e-17,-1.104399e-16,1.369126e-16,2.146097e-16
